### Домашняя работа №1.1 (Pandas)
В городе SQL произошло убийство! SQL Murder Mystery - это одновременно и самостоятельный урок для изучения концепций и команд SQL, и увлекательная игра для опытных пользователей SQL, в которой нужно раскрыть интригующее преступление.
Произошло преступление, и детективу нужна ваша помощь. Детектив дал вам отчет о месте преступления, но вы каким-то образом потеряли его. Вы смутно помните, что преступление было убийством, произошедшим 15 января 2018 года, и что оно произошло в SQL City. Начните с поиска соответствующего отчета о месте преступления в базе данных полицейского управления.

### Условия
Главное условие - решение полностью с использованием Pandas

Данные не на русском языке, так что где-то придется пользоваться переводчиком

### Критерии оценивания

1. Нашел преступника? - 4 балла
2. Использование сортировки или срезов по данным (оператор []) во время поиска - 1 балл
3. Использование фильтрации таблиц - 2 балла
4. Использование мерджей - 3 балла


In [202]:
import pandas as pd
import os

PATH_TO_DATA = './data_pandas/'


In [203]:
crime_scene_report = pd.read_csv(os.path.join(PATH_TO_DATA, 'crime_scene_report.csv'))
person = pd.read_csv(os.path.join(PATH_TO_DATA, 'person.csv'))
interview = pd.read_csv(os.path.join(PATH_TO_DATA, 'interview.csv'))
get_fit_now_member = pd.read_csv(os.path.join(PATH_TO_DATA, 'get_fit_now_member.csv'))
drivers_license = pd.read_csv(os.path.join(PATH_TO_DATA, 'drivers_license.csv'))
facebook_event_checkin = pd.read_csv(os.path.join(PATH_TO_DATA, 'facebook_event_checkin.csv'))
income = pd.read_csv(os.path.join(PATH_TO_DATA, 'income.csv'))


In [204]:
# Всем встать, суд пошел;

crime_scene_report['date'] = pd.to_datetime(crime_scene_report['date'], format='%Y%m%d')

# Убийство произошло 15 января 2018 в SQL City:
report = crime_scene_report[(crime_scene_report['date'] == '2018-01-15') &
                            (crime_scene_report['city'] == 'SQL City') &
                            (crime_scene_report['type'] == 'murder')]

print("Выписка из дела:", *report['description'], sep='\n')


Выписка из дела:
Security footage shows that there were 2 witnesses. The first witness lives at the last house on "Northwestern Dr". The second witness, named Annabel, lives somewhere on "Franklin Ave".


In [205]:
# Подшиваем к делу свидетелей;
# первый (последний дом на Northwestern Dr):

witness1 = person[person['address_street_name'] == 'Northwestern Dr'].sort_values(by='address_number', ascending=False).iloc[0]

print("Первый свидетель:", *witness1[['id', 'name']])

# второй (Анабель с Franklin Ave):
witness2 = person[(person['address_street_name'] == 'Franklin Ave') & (person['name'].str.contains('Annabel'))].iloc[0]
print("Второй свидетель:", *witness2[['id', 'name']])


Первый свидетель: 14887 Morty Schapiro
Второй свидетель: 16371 Annabel Miller


In [206]:
# Андрюха, по коням; возможно криминал: производим допрос

annabel_miller_id = person[person['name'] == 'Annabel Miller']['id'].values[0]
annabel_miller = interview[interview['person_id'] == annabel_miller_id]['transcript'].values[0]
print("Показания Аннабель:", annabel_miller, sep='\n')

# Rick?
morty_schapiro_id =  person[person['name'] == 'Morty Schapiro']['id'].values[0]
morty_schapiro = interview[interview['person_id'] == morty_schapiro_id]['transcript'].values[0]
print("Показания Морти:", morty_schapiro, sep='\n')


Показания Аннабель:
I saw the murder happen, and I recognized the killer from my gym when I was working out last week on January the 9th.
Показания Морти:
I heard a gunshot and then saw a man run out. He had a "Get Fit Now Gym" bag. The membership number on the bag started with "48Z". Only gold members have those bags. The man got into a car with a plate that included "H42W".


In [207]:
# По словам свидетелей, ищем качка из фитнес-клуба с gold статусом и номером машины, что содержит подстроку 'H42W'

suspects = get_fit_now_member[get_fit_now_member['membership_status'] == 'gold']
suspects = suspects[suspects['id'].str.contains('48Z')]
suspects = pd.merge(suspects, person, left_on='person_id', right_on='id')
suspects = pd.merge(suspects, drivers_license, left_on='license_id', right_on='id')
suspects = suspects[suspects['plate_number'].str.contains('H42W')]
print("Подозреваемый:", suspects[['name_x', 'address_number', 'address_street_name', 'plate_number']], sep='\n')


Подозреваемый:
          name_x  address_number    address_street_name plate_number
0  Jeremy Bowers             530  Washington Pl, Apt 3A       0H42W2


In [208]:
# майор Доигралес запрашивает показания:

killer = interview[interview['person_id'] == person[person['name'] == 'Jeremy Bowers']['id'].values[0]]
print("Показания подозреваемого:", *killer['transcript'], sep='\n')


Показания подозреваемого:
I was hired by a woman with a lot of money. I don't know her name but I know she's around 5'5" (65") or 5'7" (67"). She has red hair and she drives a Tesla Model S. I know that she attended the SQL Symphony Concert 3 times in December 2017.



In [209]:
# единственный подозреваемый, на которого показывают все улики, сдал наемницу с потрохами: 
# Ищем состоятельную рыжеволосую водительницу Tesla, посетившую SQL Symphony три раза и
#  стукнувшуюся столько же раз головй о проем в театре (рост 165-170 см).

# Начнем с оркестра; все, кто посещал его три раза в декабре 2017:

facebook_event_checkin['date'] = pd.to_datetime(facebook_event_checkin['date'], format='%Y%m%d')

concert_name = 'SQL Symphony Concert'
concert_date_start = '2017-12-01'
concert_date_end = '2017-12-31'

concert_clients = facebook_event_checkin[(facebook_event_checkin['event_name'] == concert_name) & 
                                          (facebook_event_checkin['date'] >= concert_date_start) & 
                                          (facebook_event_checkin['date'] <= concert_date_end)]
concert_clients_count = concert_clients['person_id'].value_counts()
frequent_clients_ids = concert_clients_count[concert_clients_count >= 3].index

print("ID ценителей искусства:", *frequent_clients_ids, sep='\n')


ID ценителей искусства:
24556
99716


In [210]:
# дальше майор Доигралес решил проверить имущество и данные в персональных данных:

frequent_clients_ids = [24556, 99716]

person_info = person[person['id'].isin(frequent_clients_ids)]
license_ids = person_info['license_id']

clients_drivers = drivers_license[drivers_license['id'].isin(license_ids)]
clients_info = pd.merge(clients_drivers, person_info, left_on='id', right_on='license_id')

print("Информация о машинах, поле и цвете волос:")
print(clients_info[['name', 'car_model', 'plate_number', 'height', 'hair_color']])


Информация о машинах, поле и цвете волос:
               name car_model plate_number  height hair_color
0  Miranda Priestly   Model S       500123      66        red


In [211]:
# отлично, почти полное совпадение. Прежде чем объявлять мадам в розыск, проверим счета и показания:

miranda_ssn = person[person['name'] == 'Miranda Priestly']['ssn'].values[0]
miranda_income = income[income['ssn'] == miranda_ssn]['annual_income'].values[0]

miranda_id = person[person['name'] == 'Miranda Priestly']['id'].values[0]
miranda_interview = interview[interview['person_id'] == miranda_id]['transcript'].values

print("Годовой доход Миранды Престли:", miranda_income, sep='\n')
print("Показания:", miranda_interview, sep='\n')


Годовой доход Миранды Престли:
310000
Показания:
[]


In [212]:
# заводим уголовку и по домам:
print("Убийца:", *suspects['name_x'].values)
print("Заказчица: ", *clients_info['name'].values)


Убийца: Jeremy Bowers
Заказчица:  Miranda Priestly
